In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [3]:
iris=load_iris()
X=iris.data[:,2:]
y=iris.target

tree_clf=DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(max_depth=2)

In [8]:
from sklearn.tree import export_graphviz

export_graphviz(
tree_clf,
out_file=image_path('iris_tree.dot'),
feature_names=iris.feature_names[2:],
class_names=iris.target_names,
rounded=True,
filled=True)

NameError: name 'image_path' is not defined

In [4]:
#클래스 확률 추정

tree_clf.predict_proba([[5,1.5]])

tree_clf.predict([[5,1.5]])

array([1])

In [5]:
#회귀

from sklearn.tree import DecisionTreeRegressor

tree_reg=DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [6]:
#7

from sklearn.datasets import make_moons

X,y=make_moons(n_samples=10000,noise=0.4,random_state=42)

In [9]:
X.shape

(10000, 2)

In [10]:
y.shape

(10000,)

In [11]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
from sklearn.model_selection import GridSearchCV

params={'max_leaf_nodes':list(range(2,100)),'min_samples_split':[2,3,4]}
grid_search_cv=GridSearchCV(DecisionTreeClassifier(random_state=42),params,verbose=1,cv=3)

grid_search_cv.fit(X_train,y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [13]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [14]:
from sklearn.metrics import accuracy_score

y_pred=grid_search_cv.predict(X_test)
accuracy_score(y_test,y_pred)

0.8695

In [16]:
from sklearn.model_selection import ShuffleSplit

n_trees=1000
n_instances=100

mini_sets=[]

rs=ShuffleSplit(n_splits=n_trees,test_size=len(X_train)-n_instances,random_state=42)
for mini_train_index,mini_test_index in rs.split(X_train):
    X_mini_train=X_train[mini_train_index]
    y_mini_train=y_train[mini_train_index]
    mini_sets.append((X_mini_train,y_mini_train))

In [19]:
from sklearn.base import clone
import numpy as np
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

In [21]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [22]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [23]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.872